# 3. Q&A에 특화된 미세 조정 모델 훈련
이 노트북은 컨텍스트, 질문 및 답변 쌍의 데이터 세트를 활용하여 해당 컨텍스트에서 질문이 생성되지 않은 적대적 질문 및 컨텍스트 쌍을 추가로 생성합니다. 이러한 경우 모델은 "질문에 대답하기에 충분한 컨텍스트가 없습니다"라고 대답하도록 프롬프트됩니다. 또한 문맥을 기반으로 질문에 대한 답변이 가능한지 여부를 예측하는 판별자 모델을 훈련할 것입니다.

우리는 의미론적으로 유사한 섹션이나 동일한 기사에서 비롯된 인접 섹션을 기반으로 하는 하드 적대적 예제도 추가할 것입니다.

In [1]:
import openai
import pandas as pd
df = pd.read_csv('olympics-data/olympics_qa.csv')
olympics_search_fileid = "file-c3shd8wqF3vSCKaukW4Jr1TT"
df.head()

,title,heading,content,tokens,context,questions,answers
0,2020 Summer Olympics,Summary,The 2020 Summer Olympics (Japanese: 2020年夏季オリン...,713,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,1. What is the 2020 Summer Olympics?\n2. When ...,1. The 2020 Summer Olympics is an internationa...
1,2020 Summer Olympics,Host city selection,The International Olympic Committee (IOC) vote...,126,2020 Summer Olympics\nHost city selection\n\nT...,1. \n2. \n3. \n4.,1. What is the International Olympic Committee...
2,2020 Summer Olympics,Impact of the COVID-19 pandemic,"In January 2020, concerns were raised about th...",369,2020 Summer Olympics\nImpact of the COVID-19 p...,1. What was the COVID-19 pandemic?\n2. How did...,1. The COVID-19 pandemic was a pandemic that o...
3,2020 Summer Olympics,Qualifying event cancellation and postponement,Concerns about the pandemic began to affect qu...,298,2020 Summer Olympics\nQualifying event cancell...,1. What was the original location of the Asia ...,1. The original location of the Asia & Oceania...
4,2020 Summer Olympics,Effect on doping tests,Mandatory doping tests were being severely res...,163,2020 Summer Olympics\nEffect on doping tests\n...,1. What was the COVID-19 pandemic?\n2. What di...,1. The COVID-19 pandemic was a pandemic that o...


섹션을 교육 및 테스트 세트로 분할

In [2]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
len(train_df), len(test_df)

(3014, 754)

사용하려는 구분자가 컨텍스트 내에 존재하지 않는지 확인합니다.

In [3]:
df.context.str.contains('->').sum()

0

## 3.1 Q&A 및 판별자 모델에 대한 미세 조정 데이터 세트 생성
미세 조정 데이터 세트는 다음과 같은 방식으로 생성됩니다. 해당하는 모든 질문, 답변 및 컨텍스트 쌍에 대해 다음을 생성합니다.
- 긍정적 예: 올바른 질문, 답변, 문맥 쌍
- 부정적인 예:
   - 임의의 컨텍스트가 질문과 쌍을 이루는 임의의 부정 예
   - 두 개의 하드 네거티브 예
     - 동일한 위키피디아 기사에서 유래한 것
     - 정확한 문맥과 가장 유사한 또 다른 것

때때로 다른 상황에서 질문에 대답할 수 있기 때문에 이 프로세스는 시끄럽지만 평균적으로 이것이 성능에 너무 많은 영향을 미치지 않기를 바랍니다.

판별자와 Q&A 응답 모델 모두에 동일한 데이터 세트 생성 프로세스를 적용합니다. 훈련 세트의 예제가 테스트 세트에 포함되지 않도록 훈련 세트와 테스트 세트에 대해 별도로 프로세스를 적용합니다.

In [4]:
import random

def get_random_similar_contexts(question, context, file_id=olympics_search_fileid, search_model='ada', max_rerank=10):
    """
    Find similar contexts to the given context using the search file
    """
    try:
        results = openai.Engine(search_model).search(
            search_model=search_model, 
            query=question, 
            max_rerank=max_rerank,
            file=file_id
        )
        candidates = []
        for result in results['data'][:3]:
            if result['text'] == context:
                continue
            candidates.append(result['text'])
        random_candidate = random.choice(candidates)
        return random_candidate
    except Exception as e:
        print(e)
        return ""

def create_fine_tuning_dataset(df, discriminator=False, n_negative=1, add_related=False):
    """
    Create a dataset for fine tuning the OpenAI model; either for a discriminator model, 
    or a model specializing in Q&A, where it says if no relevant context is found.

    Parameters
    ----------
    df: pd.DataFrame
        The dataframe containing the question, answer and context pairs
    discriminator: bool
        Whether to create a dataset for the discriminator
    n_negative: int
        The number of random negative samples to add (using a random context)
    add_related: bool
        Whether to add the related contexts to the correct context. These are hard negative examples

    Returns
    -------
    pd.DataFrame
        The dataframe containing the prompts and completions, ready for fine-tuning
    """
    rows = []
    for i, row in df.iterrows():
        for q, a in zip(("1." + row.questions).split('\n'), ("1." + row.answers).split('\n')):
            if len(q) >10 and len(a) >10:
                if discriminator:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\n Related:", "completion":f" yes"})
                else:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\nAnswer:", "completion":f" {a[2:].strip()}"})

    for i, row in df.iterrows():
        for q in ("1." + row.questions).split('\n'):
            if len(q) >10:
                for j in range(n_negative + (2 if add_related else 0)):
                    random_context = ""
                    if j == 0 and add_related:
                        # add the related contexts based on originating from the same wikipedia page
                        subset = df[(df.title == row.title) & (df.context != row.context)]
                        
                        if len(subset) < 1:
                            continue
                        random_context = subset.sample(1).iloc[0].context
                    if j == 1 and add_related:
                        # add the related contexts based on the most similar contexts according to the search
                        random_context = get_random_similar_contexts(q[2:].strip(), row.context, search_model='ada', max_rerank=10)
                    else:
                        while True:
                            # add random context, which isn't the correct context
                            random_context = df.sample(1).iloc[0].context
                            if random_context != row.context:
                                break
                    if discriminator:
                        rows.append({"prompt":f"{random_context}\nQuestion: {q[2:].strip()}\n Related:", "completion":f" no"})
                    else:
                        rows.append({"prompt":f"{random_context}\nQuestion: {q[2:].strip()}\nAnswer:", "completion":f" No appropriate context found to answer the question."})

    return pd.DataFrame(rows) 

판별자와 Q&A 응답 모델 모두에 동일한 데이터 세트 생성 프로세스를 적용합니다. 훈련 세트의 예제가 테스트 세트에 포함되지 않도록 훈련 세트와 테스트 세트에 대해 별도로 프로세스를 적용합니다.

In [5]:
for name, is_disc in [('discriminator', True), ('qa', False)]:
    for train_test, dt in [('train', train_df), ('test', test_df)]:
        ft = create_fine_tuning_dataset(dt, discriminator=is_disc, n_negative=1, add_related=True)
        ft.to_json(f'{name}_{train_test}.jsonl', orient='records', lines=True)

다음을 사용하여 사용할 수 있는 미세 조정 도구의 권장 사항에 따라 데이터 형식을 지정했습니다.
> openai 도구 fine_tunes.prepare_data -f qa_train.jsonl

미세 조정을 위한 데이터 형식 개선을 제안하는 이 도구를 사용하는 것이 좋습니다.


## 3.2 미세 조정을 위해 데이터세트 제출

In [6]:
!openai api fine_tunes.create -t "olympics-data/discriminator_train.jsonl" -v "olympics-data/discriminator_test.jsonl" --batch_size 16  --compute_classification_metrics --classification_positive_class " yes" --model ada

In [7]:
!openai api fine_tunes.create -t "olympics-data/qa_train.jsonl" -v "olympics-data/qa_test.jsonl" --batch_size 16

## 3.3 미세 조정된 모델 사용

이제 미세 조정된 판별자와 미세 조정된 Q&A 모델을 사용합니다. logprobs를 요청하면 판별자가 '예' 대 '아니오' 응답에서 얼마나 확실한지 확인할 수 있습니다.

In [8]:
ft_discriminator = "curie:ft-openai-internal-2021-08-23-23-58-57"
ft_qa = "curie:ft-openai-internal-2021-08-23-17-54-10"

def apply_ft_discriminator(context, question, discriminator_model):
    """
    Apply the fine tuned discriminator to a question, to assess whether it can be answered from the context.
    """
    prompt = f"{context}\nQuestion: {question}\n Related:"
    result = openai.Completion.create(model=discriminator_model, prompt=prompt, max_tokens=1, temperature=0, top_p=1, n=1, logprobs=2)
    return result['choices'][0]['logprobs']['top_logprobs']

apply_ft_discriminator('The first human-made object in space was the Soviet Union satellite Sputnik 1 on 4 October 1957.', 
                        'What was the first human-made object in space?', ft_discriminator)

[<OpenAIObject at 0x7fe812e602b0> JSON: {
   " no": -10.819577,
   " yes": -2.045765e-05
 }]

우리는 모델이 다른 컨텍스트와 질문에 잘 일반화될 수 있음을 알 수 있습니다.

In [9]:
def apply_ft_qa_answer(context, question, answering_model):
    """
    Apply the fine tuned discriminator to a question
    """
    prompt = f"{context}\nQuestion: {question}\nAnswer:"
    result = openai.Completion.create(model=answering_model, prompt=prompt, max_tokens=30, temperature=0, top_p=1, n=1, stop=['.','\n'])
    return result['choices'][0]['text']

apply_ft_qa_answer('The first human-made object in space was the Soviet Union satellite Sputnik 1 on 4 October 1957.', 
                    'What was the first human-made object in space?', ft_qa)


' The first human-made object in space was the Soviet Union satellite Sputnik 1 on 4 October 1957'

컨텍스트가 적절할 때 모델이 질문에 답할 수 있음을 알 수 있습니다.

In [10]:
apply_ft_qa_answer('The first human-made object in space was the Soviet Union satellite Sputnik 1 on 4 October 1957.',
                    'What is impressive about the Soviet Union?', ft_qa)

' The Soviet Union was the first country to successfully launch a satellite into space'

In [11]:
apply_ft_qa_answer('The first human-made object in space was the Soviet Union satellite Sputnik 1 on 4 October 1957.',
                    'How many cars were produced in the Soviet Union in 1970?', ft_qa)

' No appropriate context found to answer the question'

우리는 모델이 질문에 답해야 할 때와 질문에 답하기 위한 컨텍스트가 충분하지 않다고 말할 때를 알고 있음을 알 수 있습니다.

판별자와 기본 모델 또는 미세 조정된 Q&A 모델을 결합할 수도 있습니다. 판별자는 본질적으로 주어진 맥락에서 질문에 답할 수 있는지 여부를 결정하는 역할을 할 수 있습니다.

In [12]:
def answer_question_conditionally(answering_model, discriminator_model, context, question, discriminator_logprob_yes_modifier=0):
    logprobs = apply_ft_discriminator(context, question, discriminator_model)
    yes_logprob = logprobs[' yes'] if ' yes' in logprobs else -100
    no_logprob = logprobs[' no'] if ' no' in logprobs else -100
    if yes_logprob + discriminator_logprob_yes_modifier < no_logprob:
        return " No appropriate context found to answer the question based on the discriminator."
    return apply_ft_qa_answer(context, question, answering_model)
answer_question_conditionally(ft_qa, ft_discriminator, 
                                "Crowdless games are a rare although not unheard-of occurrence in sports. \
                                 When they do occur, it is usually the result of events beyond the control \
                                 of the teams or fans, such as weather-related concerns, public health concerns, \
                                 or wider civil disturbances unrelated to the game. For instance, \
                                 the COVID-19 pandemic caused many sports leagues around the world \
                                 to be played behind closed doors.",
                                "Could weather cause a sport event to have no crowd?")

' Weather could cause a sport event to have no crowd'

위의 함수는 잠재적으로 판별자와 미세 조정된 Q&A 모델을 결합하는 방법을 보여줍니다. 이렇게 하면 모델이 질문에 답하기 전에 모델이 얼마나 확실하게 되기를 원하는지 보다 세밀하게 제어할 수 있습니다.

이제 답변 엔드포인트가 작동하는 방식을 살펴보겠습니다. 검색을 결합하여 지식 기반에서 관련 컨텍스트를 검색한 다음 미세 조정된 Q&A 모델을 사용하여 질문에 답변합니다.

## 3.4 지식 기반을 기반으로 질문에 답하기
마지막으로 [/answers](https://beta.openai.com/docs/api-reference/answers) 엔드포인트와 유사한 로직을 사용할 수 있습니다. 여기서 먼저 관련 컨텍스트를 검색한 다음 Q&A 모델에 다음을 요청합니다. 주어진 맥락에서 질문에 답하십시오. 구현 세부정보를 보려면 [`answers_with_ft.py`](answers_with_ft.py) 파일을 확인하세요.

In [13]:
from answers_with_ft import answer_question
answer_question(olympics_search_fileid, ft_qa, "Which country won the Women's football tournament at the 2020 Olympic games?")

" Canada won the Women's football tournament at the 2020 Olympic games"